In [1]:
!module load nvidia/cuda/12.1.0

In [1]:
from trl import SFTTrainer

2024-04-27 22:32:52.800597: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-27 22:32:53.557789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-27 22:32:53.557982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-27 22:32:53.677247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-27 22:32:54.001442: I tensorflow/core/platform/cpu_feature_guar

In [2]:

import os
import numpy as np
import pandas as pd


from torch.utils.data import Dataset
from peft import (
    LoraConfig, 
    get_peft_model, 
)

from functools import partial
from typing import Any, Dict, List

import torch
import wandb
from datasets import DatasetDict, load_dataset
from omegaconf import DictConfig

from torch import nn

from transformers import (
    LlamaTokenizer, 
    LlamaForSequenceClassification,
    AutoTokenizer,
    Trainer, 
    TrainingArguments,
    AutoModelForSequenceClassification,
    EarlyStoppingCallback,
    TrainerCallback,
)

from structllm.models.utils import CustomWandbCallback_FineTune, EvaluateFirstStepCallback


In [3]:
IGNORE_INDEX = -100
MAX_LENGTH = 2048
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"


pretrained_ckpt = "meta-llama/Llama-2-7b-hf"

llama_tokenizer = LlamaTokenizer.from_pretrained(
    pretrained_ckpt,
    model_max_length=MAX_LENGTH,
    padding_side="right",
    use_fast=False,
    )
special_tokens_dict = dict()
if llama_tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if llama_tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if llama_tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if llama_tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

num_new_tokens = llama_tokenizer.add_special_tokens(special_tokens_dict)
llama_tokenizer.add_special_tokens(special_tokens_dict)
   


0

In [6]:
len(llama_tokenizer)

32001

In [7]:
def smart_tokenizer_and_embedding_resize(
    special_tokens_dict, 
    llama_tokenizer, 
    model,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = llama_tokenizer.add_special_tokens(special_tokens_dict)
    llama_tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(llama_tokenizer),pad_to_multiple_of=8)

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
     #   output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
     #   output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg

    model.config.pad_token_id = llama_tokenizer.pad_token_id
     #   output_embeddings[-num_new_tokens:] = output_embeddings_avg



In [8]:
from transformers import (
    AutoModelForCausalLM)

from transformers import BitsAndBytesConfig

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
device_map = {"": 0}


################################################################################
# QLoRA parameters
################################################################################


lora_r = 64
lora_alpha = 16
lora_dropout = 0.1




compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


model_name ="meta-llama/Llama-2-7b-hf"
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1



# # Load LLaMA tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)




Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
        

        

smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    llama_tokenizer=llama_tokenizer,
    model=model,
)

trainable params: 33,554,432 || all params: 6,771,970,048 || trainable%: 0.49548996469513035


In [4]:
def _tokenize(examples):
    # Tokenize the 'crystal_llm' column using the LAMA tokenizer
    tokenized_examples = llama_tokenizer(examples["crystal_llm_rep"],truncation=True, padding=True,)
    return tokenized_examples

def _prepare_datasets(path: str) -> DatasetDict:
        """
        Prepare training and validation datasets.

        Args:
            train_df (pd.DataFrame): DataFrame containing training data.

        Returns:
            DatasetDict: Dictionary containing training and validation datasets.
        """

        ds = load_dataset("json", data_files=path,split="train")
        dataset = ds.train_test_split(shuffle=True, test_size=0.2, seed=42)
        return dataset.map(_tokenize, batched=True)

In [5]:
dataset = _prepare_datasets("/work/so87pot/material_db/matbench_sml/train_matbench_log_kvrh_0.json")

Map:   0%|          | 0/4858 [00:00<?, ? examples/s]

Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['cif_p1', 'cif_symmetrized', 'cif_bonding', 'slice', 'composition', 'crystal_llm_rep', 'robocrys_rep', 'wycoff_rep', 'mbid', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 4858
    })
    test: Dataset({
        features: ['cif_p1', 'cif_symmetrized', 'cif_bonding', 'slice', 'composition', 'crystal_llm_rep', 'robocrys_rep', 'wycoff_rep', 'mbid', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1215
    })
})

In [14]:
REPRESENTATION_MAP ={
    "cif_p1" : "cif_p1",
    "Slice"  : "slice",
}


PROPERTY_MAP = {
    "matbench_log_gvrh" : "shear modulus in GPa",
    "matbench_log_kvrh" : "bulk modulus in GPa",
    "matbench_dielectric" : "refractive index",
    "matbench_perovskites" : "formation energy in eV",
    
}

MATERAIL_MAP = {
    "matbench_log_gvrh" : "material",
    "matbench_log_kvrh" : "material",
    "matbench_dielectric" : "dielectric material",
    "matbench_perovskites" : "perovskite material",    
}


def format_qstns(sample,representation):
    property = "shear modulus" 
    material = "material"
    question = f"<s>Question: What is the {property} of the {material} "
    material = f"{sample[representation]}" 
    response = f" Answer: {round(float(sample['labels']),3)}"
    # join all the parts together
    prompt = "".join([i for i in [question,material, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset_test(sample,representation="slice"):
    sample["text"] = f"{format_qstns(sample,representation)}{llama_tokenizer.eos_token}"
    return sample




In [15]:

trainset = dataset['train'].map(template_dataset_test)
testset= dataset['test'].map(template_dataset_test)

Map:   0%|          | 0/4858 [00:00<?, ? examples/s]

Map:   0%|          | 0/1215 [00:00<?, ? examples/s]

In [16]:
trainset[0]['text']

'<s>Question: What is the shear modulus of the material Zn Co O O 0 2 o - o 0 2 - o o 0 2 o o - 0 3 - - o 0 3 o - - 0 3 - o - 1 3 o - o 1 3 - o o 1 3 o o - 1 2 o o + 1 2 + o o 1 2 o + o  Answer: 2.23</s>'

In [20]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 8
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True


per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True

max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25


# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

max_seq_length = None
packing = False
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    train_dataset=trainset,
    eval_dataset=testset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=llama_tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("./")

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacty of 39.43 GiB of which 43.75 MiB is free. Including non-PyTorch memory, this process has 39.31 GiB memory in use. Of the allocated memory 36.03 GiB is allocated by PyTorch, and 1.87 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

In [ ]:
def format_dolly(sample):
    instruction = f"<s>[INST] {sample['instruction']}"
    context = f"Here's some context: {sample['context']}" if len(sample["context"]) > 0 else None
    response = f" [/INST] {sample['response']}"
    # join all the parts together
    prompt = "".join([i for i in [instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{llama_tokenizer.eos_token}"
    return sample

In [ ]:
# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
dataset = dataset_shuffled.select(range(50))

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset